# S&P 500 Equal Weight Index Fund
- The most popular of the all the index funds is the S&P 500
- It contains the largest 500 (or so) companies of the US of A

## Goal
- The goal of this script is to create a python script which will accept the value of your portfolio, telling you the total amount of shares each S&P 500 memebr you should buy to achieve an equal weighted version
    - Equal weighted meaning the stock are guaged on size rather than market capitalization

In [48]:
# Imports 
import numpy as np
import pandas as pd
import requests
import xlsxwriter as xls
from dotenv import dotenv_values
import math
import json
import time

# config = {"USER": "foo", "EMAIL": "foo@example.org"}
env = dotenv_values(".env")  

In [49]:
# Importing our list of the top tokens from coin gecko
topTokens = pd.read_csv("top_101_tokens.csv")
topTokens

,Ticker,Name,ID
0,btc,Bitcoin,bitcoin
1,eth,Ethereum,ethereum
2,usdt,Tether,tether
3,bnb,BNB,binancecoin
4,sol,Solana,solana
...,...,...,...
96,flr,Flare,flare-networks
97,usdd,USDD,usdd
98,frxeth,Frax Ether,frax-ether
99,gala,GALA,gala


In [50]:
# Adding price, market cap and how much tokens to buy to our dataframe
my_columns = ['Token Price', 'Market Cap', 'No. To Buy']
new_df = pd.DataFrame(columns=my_columns)
new_df

,Token Price,Market Cap,No. To Buy


In [51]:
final_dataframe = pd.concat([topTokens, new_df])
final_dataframe

,Ticker,Name,ID,Token Price,Market Cap,No. To Buy
0,btc,Bitcoin,bitcoin,NaN,NaN,NaN
1,eth,Ethereum,ethereum,NaN,NaN,NaN
2,usdt,Tether,tether,NaN,NaN,NaN
3,bnb,BNB,binancecoin,NaN,NaN,NaN
4,sol,Solana,solana,NaN,NaN,NaN
...,...,...,...,...,...,...
96,flr,Flare,flare-networks,NaN,NaN,NaN
97,usdd,USDD,usdd,NaN,NaN,NaN
98,frxeth,Frax Ether,frax-ether,NaN,NaN,NaN
99,gala,GALA,gala,NaN,NaN,NaN


In [58]:
# Split the IDs into chunks of 10 to respect the rate limit
chunks = [final_dataframe['ID'][i:i + 10] for i in range(0, final_dataframe.shape[0], 10)]

# Add columns for Token Price and Market Cap
final_dataframe['Token Price'] = pd.Series(dtype=float)
final_dataframe['Market Cap'] = pd.Series(dtype=float)

# Fetch data in batches
for chunk in chunks:
    ids = ','.join(chunk)
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies=usd&include_market_cap=true"
    response = requests.get(url)
    data = response.json()
    for coin_id in chunk:
        if coin_id in data:
            final_dataframe.loc[final_dataframe['ID'] == coin_id, 'Token Price'] = data[coin_id]['usd']
            final_dataframe.loc[final_dataframe['ID'] == coin_id, 'Market Cap'] = data[coin_id]['usd_market_cap']
    time.sleep(1)  # Sleep to respect the rate limit of 10 calls per second

In [62]:
# Removing the all the tokens which came up blank for both market cap and price
final_dataframe = final_dataframe.iloc[:50]

In [73]:
# Trying to ascertain the portfolio size of an individual
# To know how much they would need to allocate for their tokens based on the options available 
portfolio_size = input('Enter the value of your portfolio')
try:
    portfolio_val = float(portfolio_size)
except ValueError:
    portfolio_size = input('Enter the value of your portfolio')
    portfolio_val = float(portfolio_size)


In [77]:
# Here we trying to figure out the position size needed for the portfolio allocation based on the total number of tokens available to be bought (top 50 according to coin gecko)
position_size = portfolio_val / (len(final_dataframe.index))

for eachIndex in range(0, len( final_dataframe.index)):
    final_dataframe.loc[eachIndex, 'No. To Buy'] = math.floor(position_size/final_dataframe.loc[eachIndex, 'Token Price'])

,Ticker,Name,ID,Token Price,Market Cap,No. To Buy
0,btc,Bitcoin,bitcoin,43029.000000,8.445797e+11,0
1,eth,Ethereum,ethereum,2301.390000,2.767298e+11,8
2,usdt,Tether,tether,0.999741,9.619432e+10,20005
3,bnb,BNB,binancecoin,300.760000,4.628240e+10,66
4,sol,Solana,solana,97.480000,4.249189e+10,205
5,xrp,XRP,ripple,0.507940,2.761227e+10,39374
6,usdc,USDC,usd-coin,0.999583,2.675463e+10,20008
7,steth,Lido Staked Ether,staked-ether,2303.750000,2.176629e+10,8
8,ada,Cardano,cardano,0.502002,1.763100e+10,39840
9,avax,Avalanche,avalanche-2,33.780000,1.241339e+10,592


In [ ]:
# Outputting the data file to a excel file
writer = pd.ExcelWriter('recommnded_trade.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)